# Vision Fine Tune GPT-4o

In [1]:
from openai import OpenAI
import os
import json
import pandas as pd
import dotenv
from dotenv import load_dotenv
import gpt_utils
from pydantic import BaseModel

In [2]:
proj_dir = f'/zfs/projects/darc/nrapstin_hht_image_ai'

In [3]:
dotenv.load_dotenv(f'{proj_dir}/.env', override=True)

True

In [4]:
api_key = os.environ["OPENAI_API_KEY"]

In [5]:
client = OpenAI()

In [6]:
class CarYear(BaseModel):
    year: int

## Load data

In [7]:
data_file = f"{proj_dir}/stanford-cars/data/train.csv"

In [8]:
df = pd.read_csv(data_file, usecols=['image', 'Class', 'Class Name'])

### Prepare train, validation files

In [9]:
gpt_utils.prepare_ft_data(df)

JSONL files already exist. Skipping creation.


### The limits for jsonl with images:
- Your training file can contain a maximum of 50,000 examples that contain images (not including text examples).
- Each example can have at most 10 images.
- Each image can be at most 10 MB.

- Randomly select 15 images per class for training, 10 per class for validation, and 10 per class for testing.
- Create three JSONL files (`train.jsonl`, `val.jsonl`, and `test.jsonl`) using the provided `create_jsonl` function.

## Fine-Tune

In [ ]:
model = "gpt-4o-2024-08-06"

In [ ]:
jsonl_train_file = 'data/fine-tune/train.jsonl'
jsonl_val_file = 'data/fine-tune/val.jsonl'

In [ ]:
train_file = client.files.create(
  file=open(jsonl_train_file, "rb"),
  purpose="fine-tune"
)

In [ ]:
val_file = client.files.create(
  file=open(jsonl_val_file, "rb"),
  purpose="fine-tune"
)

See uploaded jsonl files at https://platform.openai.com/storage/files

In [ ]:
# start fine-tuning job
ft_job = client.fine_tuning.jobs.create(
         training_file = train_file.id,
         validation_file = val_file.id,
         model = model
)

## Predict using GPT-4o Model using Batches API

First, run `create_test_jsonl_base.py`.

In [ ]:
test_file = f'{proj_dir}/stanford-cars/gpt/data/fine-tune/test_base_model.jsonl'

In [ ]:
batch_input_file = client.files.create(
  file=open(test_file, "rb"),
  purpose="batch"
)

See uploaded jsonl files at https://platform.openai.com/storage/files

In [ ]:
response = client.batches.create(
            input_file_id = batch_input_file.id,
            endpoint = "/v1/chat/completions",
            completion_window = "24h",
            metadata = {
              "description": f"Image label prediction for test set using the base model"
                })

In [ ]:
# Save the base model batch ID
gpt_utils.save_batch_id(response.id, "response_id_base_model_test_set")

- After submitting to Batches API, go to https://platform.openai.com/batches
- Wait until the batch is complete. Then get the responses.

In [ ]:
# Later, when you want to load the batch IDs:
base_model_batch_id = gpt_utils.load_batch_id("response_id_base_model_test_set")

In [ ]:
print(f"Base Model Batch ID: {base_model_batch_id}")

In [ ]:
response_out = client.batches.retrieve(base_model_batch_id)

In [ ]:
response_out

In [ ]:
jsonl_string = client.files.retrieve_content(response_out.output_file_id)

In [ ]:
# Split the string by newline characters to get each JSON line
json_lines = jsonl_string.strip().split('\n')

In [ ]:
# Parse each JSON line into a Python dictionary
json_objects = [json.loads(line) for line in json_lines]

# Load the list of JSON objects into a DataFrame
df_test = pd.DataFrame(json_objects)

In [ ]:
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv('results/responses_base_model.csv', index=False)

## Batch expired.. so we make syncronous calls instead

In [ ]:
model = "gpt-4o-2024-08-06"

In [ ]:
test_image_paths_file = f'{proj_dir}/stanford-cars/gpt/data/fine-tune/test_image_paths.txt'

In [ ]:
with open(test_image_paths_file, 'r') as f:
    test_image_paths = [line.strip() for line in f.readlines()]

In [ ]:
len(test_image_paths)

In [ ]:
%%time

responses = []
image_ids = []

for i in range(len(test_image_paths)):
# for i in range(1):

    test_image = test_image_paths[i]
    print(test_image)
    image_id = int(test_image.split('/')[-1][:-4])
    print(image_id)
    image_ids.append(image_id)
    encoded_image = gpt_utils.encode_image(test_image)
    image_content = {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{encoded_image}"}}

    completion = client.beta.chat.completions.parse(
        model = model,
        messages = [  {"role": "user", "content": [{
                            "type": "text",
                            "text": "What is the year of the car in the image?" },
                        image_content
                    ]}],
        response_format = CarYear)
    
    response = completion.choices[0].message.parsed
    year = response.year
    print(year)
    responses.append(year)

In [ ]:
len(responses)

In [ ]:
len(image_ids)

In [ ]:
df_test_out = pd.DataFrame({'Prediction': responses, 
                           'image_id': image_ids})

In [ ]:
df_test_out.head()

In [ ]:
df_test_out.to_csv(f'{proj_dir}/stanford-cars/gpt/results/base_model_test_df_responses.csv', index=False)


## Predict using Fine-Tuned Model and Batches API

First, run `create_test_jsonl_fine_tuned.py`.

In [ ]:
test_file = 'data/fine-tune/test_ft_model.jsonl'

In [ ]:
batch_input_file = client.files.create(
  file=open(test_file, "rb"),
  purpose="batch"
)

See uploaded jsonl files at https://platform.openai.com/storage/files

In [ ]:
response = client.batches.create(
            input_file_id = batch_input_file.id,
            endpoint = "/v1/chat/completions",
            completion_window = "24h",
            metadata = {
              "description": f"Image label prediction for test set using the fine-tuned model"
                })

In [ ]:
# Save the base model batch ID
gpt_utils.save_batch_id(response.id, "response_id_ft_model_test_set")

- After submitting to Batches API, go to https://platform.openai.com/batches
- Wait until the batch is complete. Then get the responses.

In [ ]:
# Later, when you want to load the batch IDs:
ft_model_batch_id = gpt_utils.load_batch_id("response_id_ft_model_test_set")

In [ ]:
print(f"Fine-tuned Model Batch ID: {ft_model_batch_id}")

In [ ]:
# response_out = client.batches.retrieve(response.id)
response_out = client.batches.retrieve(ft_model_batch_id)

In [ ]:
response_out

In [ ]:
jsonl_string = client.files.retrieve_content(response_out.output_file_id)

In [ ]:
# Split the string by newline characters to get each JSON line
json_lines = jsonl_string.strip().split('\n')

In [ ]:
# Parse each JSON line into a Python dictionary
json_objects = [json.loads(line) for line in json_lines]

# Load the list of JSON objects into a DataFrame
df_test = pd.DataFrame(json_objects)

In [ ]:
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv('results/responses_ft_model.csv', index=False)